## Loading Data 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/used-car-dataset-ford-and-mercedes/bmw.csv')

# Exploring Data

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df['transmission'].value_counts()

In [ ]:
df['fuelType'].value_counts()

In [ ]:
df['model'].value_counts()

# Visualizing Data

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt 
plt.figure(figsize=(12,10))
ax = sns.heatmap(df.corr())

In [ ]:
fig = plt.figure(figsize = (20,15))
ax = fig.gca()
df.hist(ax=ax)
plt.show()

## One Hot Encoding 

In [ ]:
df = pd.get_dummies(data=df, columns=['model','transmission','fuelType'])

In [ ]:
df.shape

# Split Data 

In [ ]:
X = df.drop('price',axis=1)
y = df['price']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.33)

# Model One-> Ridge

In [ ]:
from sklearn.metrics import mean_squared_error as MSE 
from sklearn.linear_model import Ridge

In [ ]:
model_1 = Ridge(alpha=1.0)
model_1.fit(X_train, y_train)

In [ ]:
model_1.score(X_test,y_test)

In [ ]:
y_pred = model_1.predict(X_test) 
np.sqrt(MSE(y_test, y_pred)) 

# Model Two -> Xgboost

In [ ]:
import xgboost as xg 

In [ ]:
model_2 = xg.XGBRegressor(objective ='reg:linear', 
                  n_estimators = 10, seed = 123) 
  
model_2.fit(X_train, y_train) 


In [ ]:
model_2.score(X_test,y_test)

In [ ]:
y_pred = model_2.predict(X_test) 
  
np.sqrt(MSE(y_test, y_pred)) 

# Model Three -> ElasticNet

In [ ]:
from sklearn.linear_model import ElasticNet
model_3 = ElasticNet(random_state=0)
model_3.fit(X_train, y_train)

In [ ]:
y_pred = model_3.predict(X_test)   
np.sqrt(MSE(y_test, y_pred)) 

#  Model Four -> NN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

In [ ]:
model = Sequential()

model.add(Dense(38,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse')

In [ ]:
model.fit(x=X_train, 
          y=y_train, 
          epochs=600,
          validation_data=(X_test, y_test),
          callbacks=[early_stop]
          )

In [ ]:
losses = pd.DataFrame(model.history.history)
losses.plot()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
np.sqrt(MSE(y_test,y_pred))